In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd
from tqdm import tqdm
import numpy as np
import json
link_companies = 'Users/'
df = pd.read_json('ner_output_first.json')
import pickle

/opt/anaconda3/envs/clean2/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
fuzz.ratio("filA", "ChickfilA")

62

In [2]:
ORG = []
PER = []
LOC = []
MISC = []

### Semi-Manual Part Starts

#### Misspelling corrections with fuzzy matching
We Start with the most important - organizations category

In [5]:
df['ner_output_first'][11]

[{'ORG': 'Chick Fil - A'}, {'MISC': 'Christian'}, {'MISC': 'Christian'}]

Lets get rid of MISC varirables in organizations

In [ ]:
import re

# matches exactly one letter (with no other letters around it)
single_letter = re.compile(r'^[^A-Za-z]*[A-Za-z][^A-Za-z]*$')
# matches any character that is NOT A–Z or a–z
non_alpha      = re.compile(r'[^A-Za-z]')

for row in df['ner_output_first']:
    if not isinstance(row, list) or not row:
        continue

    for entity in row:
        for label, text in list(entity.items()):
            # remove all non-letters
            cleaned = non_alpha.sub("", text)

            # if what's left is empty or just one letter, drop it
            if not cleaned or single_letter.match(cleaned):
                entity.pop(label)
            else:
                # otherwise overwrite in-place with the cleaned string
                entity[label] = cleaned

        # skip empty entities
        if not entity:
            continue

        # your existing ORG→MISC recategorization
        if entity.get('ORG') in {
            'LGBTQ', 'Pride', 'RacialJustice',
            'RighttoPrivacy', 'TV', 'Project', 'BLM', 'BlackLiveMatter', 'GunRights', 'Idk', 'LBGQT', 'MAGA', 'Matter', 'Planet'
        }:
            entity['MISC'] = entity.pop('ORG')

In [4]:
for row in df['ner_output_first']:
        if row != "[]":
            for entity in row:
                for key, value in entity.items():
                    if key == 'ORG':
                        ORG.append(value)
                    elif key == 'PER':
                        PER.append(value)
                    elif key == 'LOC':
                        LOC.append(value)
                    elif key == 'MISC':
                        MISC.append(value)

In [6]:
set(ORG)

{'AARP',
 'AB',
 'AMAC',
 'Amazon',
 'AmericanEagle',
 'AmericanVets',
 'AnheuserBusch',
 'Apple',
 'Aveda',
 'Axiology',
 'BET',
 'BLQK',
 'BassPro',
 'BassProShops',
 'BenJerrys',
 'BenJerrysIceCream',
 'Benand',
 'BenandJerry',
 'BenandJerrys',
 'BenandJerrysIceCream',
 'Bissell',
 'BlackLivesMatter',
 'BlackRifle',
 'BlackRifleCoffee',
 'BlackoutCoffee',
 'Bombas',
 'BombasCharitableSocks',
 'BrewGreen',
 'BudLight',
 'BudLite',
 'Budweiser',
 'ChicFilA',
 'ChicFila',
 'Chicfil',
 'Chicfila',
 'Chick',
 'ChickFiLe',
 'ChickFilA',
 'ChickFila',
 'ChickFilet',
 'Chickfil',
 'ChickfilA',
 'Chickfila',
 'Chik',
 'ChikFil',
 'ChikFilA',
 'ChikFilet',
 'ChikfilA',
 'Chipotle',
 'Christina',
 'CocaCola',
 'Coke',
 'Columbia',
 'Control',
 'Converse',
 'DailyWire',
 'DairyQueen',
 'DemocraticParty',
 'DicksSportingGoods',
 'Disney',
 'EFF',
 'Earth',
 'EarthFare',
 'EcoGoods',
 'EcoWear',
 'Ecosia',
 'ElectronicFrontierFoundation',
 'Elf',
 'Elon',
 'Etsy',
 'Facebook',
 'FairTrade',
 'Foc

In [8]:
df['ner_output_first'][72]

[{'ORG': 'HobbyLobby'}, {'ORG': 'Chick'}, {'MISC': 'fil'}, {}]

In [9]:
correct = ["Patagonia", "ChickfilA", "McDonalds", "LGBTQ", "Tesla", "CocaCola", 
           "Levis", "BenJerrys", "BudLight", "InNOutBurger", "LOreal", 
           "TOMS", "Amazon", "Bombas", "BlackLivesMatter", "JohnsonJohnson", 
           "Greenpeace", "Dicks Sporting Goods", "Fox", "NewmansOwn", "Rothys", "HenrySchein"]

for row in df['ner_output_first']:
    if row != "[]":
        for entity in row:
            for key, value in entity.items():
                for label_true in correct:
                    if fuzz.ratio(value, label_true) >= 65:
                        entity[key] = label_true
                    elif value in ['Fil', 'fil', 'filA']:
                        entity[key] = 'ChickfilA'
                    elif value in ['iPhones']:
                        entity[key] = 'Apple'
    else:
        continue

In [11]:
org_values = set(pd.Series(ORG))

for row in df['ner_output_first']:
    if not isinstance(row, list) or not row:
        continue

    for entity in row:
        # if it already has an ORG, skip
        if 'ORG' in entity:
            continue

        for label, text in list(entity.items()):
            # if text is in your org_values, reclassify & stop
            if text in org_values:
                entity['ORG'] = entity.pop(label)

In [13]:
for row in df['ner_output_first']:
        if row != "[]":
            for entity in row:
                for key, value in entity.items():
                    if key == 'ORG':
                        ORG.append(value)
                    elif key == 'PER':
                        PER.append(value)
                    elif key == 'LOC':
                        LOC.append(value)
                    elif key == 'MISC':
                        MISC.append(value)

In [ ]:
ORG_set = set(ORG)

{'AARP',
 'AB',
 'AMAC',
 'Amazon',
 'AmericanEagle',
 'AmericanVets',
 'AnheuserBusch',
 'Apple',
 'Aveda',
 'Axiology',
 'BET',
 'BLQK',
 'BassPro',
 'BassProShops',
 'BenJerrys',
 'BenJerrysIceCream',
 'Benand',
 'BenandJerry',
 'BenandJerrys',
 'BenandJerrysIceCream',
 'Bissell',
 'BlackLivesMatter',
 'BlackRifle',
 'BlackRifleCoffee',
 'BlackoutCoffee',
 'Bombas',
 'BombasCharitableSocks',
 'BrewGreen',
 'BudLight',
 'BudLite',
 'Budweiser',
 'ChicFilA',
 'ChicFila',
 'Chicfil',
 'Chicfila',
 'Chick',
 'ChickFiLe',
 'ChickFilA',
 'ChickFila',
 'ChickFilet',
 'Chickfil',
 'ChickfilA',
 'Chickfila',
 'Chik',
 'ChikFil',
 'ChikFilA',
 'ChikFilet',
 'ChikfilA',
 'Chipotle',
 'Christina',
 'CocaCola',
 'Coke',
 'Columbia',
 'Control',
 'Converse',
 'DailyWire',
 'DairyQueen',
 'DemocraticParty',
 'Dicks Sporting Goods',
 'DicksSportingGoods',
 'Disney',
 'EFF',
 'Earth',
 'EarthFare',
 'EcoGoods',
 'EcoWear',
 'Ecosia',
 'ElectronicFrontierFoundation',
 'Elf',
 'Elon',
 'Etsy',
 'Faceb

In [ ]:
# let's test the similarity ratio
fuzz.ratio('BlackoutCoffee', 'BlackRifleCoffee')

73

interesting ! fuzz.ratio('BlackoutCoffee', 'BlackRifleCoffee') gives 73
fuzz.ratio('Amazon', 'Patagonia') gives 40

In [16]:
to_delete_values = ['and', 'la', 'lay', 'njalla', 'pro']

for row in df['ner_output_first']:
    # skip non-lists
    if not isinstance(row, list):
        continue

    for entity in row:
        # iterate over a snapshot of the dict’s items
        for key, val in list(entity.items()):
            if val in to_delete_values:
                entity.pop(key)

In [17]:
ORG, PER, LOC, MISC = [], [], [], []

for row in df['ner_output_first']:
        if row != "[]":
            for entity in row:
                for key, value in entity.items():
                    if key == 'ORG':
                        ORG.append(value)
                    elif key == 'PER':
                        PER.append(value)
                    elif key == 'LOC':
                        LOC.append(value)
                    elif key == 'MISC':
                        MISC.append(value)

In [18]:
list_misc = ['BLM', 'BlackLivesMatter', 'GunRights', 'Idk', 'LBGQT', 'LGBTQ', 'MAGA', 'Matter', 'Planet', 'RacialJustice', 'RighttoPrivacy']
list_per = ['Christina', 'Elon', 'joeJohns']
list_loc = ['NorCal']

for row in df['ner_output_first']:
    if not isinstance(row, list) or not row:
        continue

    for entity in row:
        if not entity:
            continue

        # your existing ORG→MISC recategorization
        if entity.get('ORG') in {
            'BLM', 'BlackLivesMatter', 'GunRights', 'Idk', 'LBGQT', 'LGBTQ', 'MAGA', 'Matter', 'Planet', 'RacialJustice', 'RighttoPrivacy'
        }:
            entity['MISC'] = entity.pop('ORG')
        
        elif entity.get('ORG') in {
            'Christina', 'Elon', 'joeJohns'
        }:
            entity['PER'] = entity.pop('ORG')
        
        elif entity.get('ORG') in {
            'NorCal'
        }:
            entity['LOC'] = entity.pop('ORG')

        elif entity.get('MISC') in {
            'Grubhub', 'Tinder', 'Instagram', 'Fenty', 'Formula', 'Olympic', 'OlympicCommittee', 'Olympics'
        }:
            entity['ORG'] = entity.pop('MISC')

In [19]:
ORG, PER, LOC, MISC = [], [], [], []

for row in df['ner_output_first']:
        if row != "[]":
            for entity in row:
                for key, value in entity.items():
                    if key == 'ORG':
                        ORG.append(value)
                    elif key == 'PER':
                        PER.append(value)
                    elif key == 'LOC':
                        LOC.append(value)
                    elif key == 'MISC':
                        MISC.append(value)

In [20]:
set(PER)

{'Ben',
 'BenSapiro',
 'Budwieser',
 'Christina',
 'ColinKaepernick',
 'ColinKapernick',
 'DonaldTrump',
 'Dylan',
 'DylanMulvaney',
 'Elon',
 'ElonMusk',
 'GeorgeFloyd',
 'God',
 'Grubhub',
 'Hodgetwins',
 'Jesus',
 'Jude',
 'Kimkardashian',
 'Linda',
 'Rihanna',
 'Sabbth',
 'Tinder',
 'Tom',
 'Trump',
 'Vince',
 'VinceMcMahon',
 'joeJohns'}

We seem to have a solid ORG set. Lets remove any ORG misclassified in other categories.

In [ ]:
ORG_set = set(ORG)

def reassign_org(entities):
    # if its not a list, just return it unchanged
    if not isinstance(entities, list):
        return entities

    for ent in entities:
        if not isinstance(ent, dict):
            continue

        # iterate over a copy of (key, value) pairs so we can pop safely
        for key, value in list(ent.items()):
            if key != 'ORG' and value in ORG_set:
                ent['ORG'] = ent.pop(key)
                break

    return entities

df['ner_output_first'] = df['ner_output_first'].apply(reassign_org)

In [22]:
ORG, PER, LOC, MISC = [], [], [], []

for row in df['ner_output_first']:
        if row != "[]":
            for entity in row:
                for key, value in entity.items():
                    if key == 'ORG':
                        ORG.append(value)
                    elif key == 'PER':
                        PER.append(value)
                    elif key == 'LOC':
                        LOC.append(value)
                    elif key == 'MISC':
                        MISC.append(value)

In [24]:
set_ORG = set(ORG)
set_LOC = set(LOC)
set_PER = set(PER)
set_MISC = set(MISC)

In [ ]:
categories = [ORG, LOC, PER, MISC]

Issues: 
- Same org with two different names: ChickfilA and Chick etc.
- Values or MISC categorized as ORG

In [26]:
ORG_series = pd.Series(ORG)
ORG_series.value_counts().head(10)

ChickfilA     55
Patagonia     35
Target        24
BenJerrys     22
Starbucks     22
Nike          19
HobbyLobby    13
BudLight      11
Walmart       11
Amazon        10
Name: count, dtype: int64

In [ ]:
link = 'documents.xlsx'
df_full = pd.ExcelFile(link).parse('general_labeled')
df_full.columns = df_full.iloc[0]
df_full = df_full[1:].reset_index(drop=True)

In [28]:
df_full['ner_output'] = df['ner_output_first']

In [29]:
df_full = df_full[['recognition_value', 'rv_label', 'ner_output']]

In [30]:
df_full.to_excel('df_full.xlsx')

In [ ]:
with open('df_ner_analysis.pkl','wb') as f:
    pickle.dump(df_full, f)